In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer

def delete_hashs(df: pd.DataFrame) -> pd.DataFrame:
    df = df[~df['text'].astype(str).str.startswith('#')]
    df = df.reset_index(drop=True)
    return df

def encode_labels(df: pd.DataFrame) -> pd.DataFrame:
    for col in ['Joy', 'Trust', 'Anticipation', 'Surprise', 'Fear','Sadness', 'Disgust', 'Anger', 'Positive', 'Negative', 'Neutral']:
        df[col] = df[col].apply(lambda x: 1 if x else 0)
    return df

def embed_text(df: pd.DataFrame) -> pd.DataFrame:
    model = SentenceTransformer('sdadas/st-polish-paraphrase-from-distilroberta')
    
    corpus = [str(df.loc[index, 'text']) for index in df.index.to_list()]
    embeddings = model.encode(corpus)

    embedding_column_names = [f'embedding_{i}' for i in range(embeddings.shape[1])]
    
    df_embeddings = pd.DataFrame(embeddings, columns=embedding_column_names)
    df = pd.concat([df_embeddings, df], axis=1)
    return df

def rename_columns(df: pd.DataFrame) -> pd.DataFrame:
    df.columns = df.columns.str.lower()
    return df

def transform_texts(df: pd.DataFrame) -> pd.DataFrame:    
    result_df = delete_hashs(df=df)
    result_df = encode_labels(df=result_df)
    
    result_df = embed_text(df=result_df)
    result_df = rename_columns(df=result_df)
    
    return result_df

def transform_sentences(df: pd.DataFrame) -> pd.DataFrame:
    result_df = pd.DataFrame(data={
        'text': [],
        'Joy': [], 'Trust': [], 'Anticipation': [], 'Surprise': [], 'Fear': [], 'Sadness': [],
           'Disgust': [], 'Anger': [], 'Positive': [], 'Negative': [], 'Neutral': []
        }
    )
    
    sentences = []
    
    for index in df.index.tolist():
        if (str)(df.loc[index, 'text']).startswith('#'):
            sentence = " ".join(sentences)
            
            df.loc[index, 'text'] = sentence
            
            result_df = pd.concat([result_df, df.loc[[index]]])
            
            sentences = []
        else:
            sentences.append((str)(df.loc[index, 'text']))
            
    result_df = encode_labels(df=result_df)
    result_df = embed_text(df=result_df)
    result_df = rename_columns(df=result_df)
        
    return result_df

In [2]:
import os
from typing import List

def load_data() -> List:
    data = []
    
    for name in ['train', 'val', 'test']:
        for category in ['texts', 'sentences']:
            if os.path.exists(f'../data/clean/{name}_{category}.csv'):
                df = pd.read_csv(f'../data/clean/{name}_{category}.csv', index_col=0)
            else:
                df = pd.read_csv(f'../data/raw/{name}.csv')
                if category == 'texts':
                    df = transform_texts(df=df)
                elif category == 'sentences':
                    df = transform_sentences(df=df)
                df.to_csv(f'../data/clean/{name}_{category}.csv')

            data.append(df)        
    return data

In [3]:
data = load_data()
train_texts = data[0]
train_sentences = data[1]
val_texts = data[2]
val_sentences = data[3]
test_texts = data[4]
test_sentences = data[5]

In [4]:
train_texts.head(10)

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,trust,anticipation,surprise,fear,sadness,disgust,anger,positive,negative,neutral
0,0.625319,-0.344142,-0.336505,-0.795379,-1.198253,0.767163,0.131462,0.499302,-0.456944,1.081408,...,0,0,1,0,1,0,1,0,1,0
1,-0.076731,-0.189474,-0.241794,-1.031343,-0.410032,0.015206,-0.315487,-1.140840,0.240389,-0.044278,...,0,0,0,0,1,1,1,0,1,0
2,-0.549568,0.292970,-0.237811,-1.464507,-0.256398,-0.257640,0.129297,0.090897,0.498563,0.079539,...,0,0,0,0,0,1,1,0,1,0
3,-0.421062,-0.050713,-0.098899,-0.435932,0.217081,-0.789393,-0.019077,0.292721,-0.072796,0.310624,...,0,0,0,0,1,1,1,0,1,0
4,-0.578499,-0.247815,0.050328,-0.520087,-0.639798,0.165438,-0.473650,-0.533067,0.628907,-0.323319,...,0,0,0,0,0,0,1,0,1,1
5,-0.483665,-0.207571,-0.129095,-0.288924,0.140756,-0.087018,-0.213280,-0.585768,-0.102779,0.336518,...,0,0,0,0,0,0,0,1,0,1
6,0.093173,0.184847,-0.214419,0.295365,-0.065878,-0.215041,0.103282,-0.644195,-0.254638,0.262836,...,0,0,0,0,0,0,0,1,0,1
7,0.060029,-0.126981,-0.276597,-0.465079,-0.291366,-0.139862,-0.205201,-0.957109,-0.232539,-0.049944,...,0,0,1,0,1,0,0,0,1,0
8,0.304022,0.000443,-0.028547,-0.072599,0.057307,-0.013533,0.363386,0.208481,-0.241232,-0.392094,...,0,0,0,0,0,0,1,1,1,1
9,0.259482,0.225580,0.062550,-0.057306,0.129021,-0.502995,0.005430,0.135867,0.111791,0.000536,...,0,0,0,0,0,0,1,0,1,0


In [5]:
train_sentences.head(10)

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,trust,anticipation,surprise,fear,sadness,disgust,anger,positive,negative,neutral
0,-0.256515,-0.394120,-0.061789,-0.628941,-0.158244,-0.373374,-0.181903,-0.210515,0.250537,-0.268037,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.259670,-0.145451,0.207794,0.265693,-0.726777,-0.292627,0.072010,-0.084182,-0.181870,0.105428,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.070523,0.004500,0.106811,-0.058817,-0.322666,-0.295914,0.598172,-0.643535,-0.007154,-0.193393,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.746101,0.507065,-0.020439,-0.057654,-0.599484,-0.328928,0.145844,-0.002598,0.466886,0.119020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.052701,-0.231508,-0.161393,0.392392,-0.018702,-0.023502,-0.469076,0.000928,0.089646,0.189956,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-0.051234,0.001332,-0.207326,-0.573515,0.221425,-0.243563,-0.093504,-0.403796,-0.150234,-0.198166,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.340573,0.535078,-0.005093,-0.060553,-0.725296,-0.414659,0.058137,-0.297488,-0.530921,-0.272161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,-0.041362,-0.141436,0.095393,0.006156,-0.328054,0.150045,-0.380772,0.307239,-0.077633,0.165795,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,-0.150252,0.048888,0.083734,-0.146957,-0.934344,0.015952,0.046441,-0.191632,-0.385218,0.028283,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.210382,0.247159,-0.086737,0.202080,0.010629,-0.580839,-0.317237,-0.049054,-0.155974,0.463064,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
X_train_texts = train_texts.iloc[:, :768] # + train_sentences.iloc[:, :768]
y_train_texts = train_texts.iloc[:, 769:] # + train_sentences.iloc[:, 769:]

X_val_texts = val_texts.iloc[:, :768] # + val_sentences.iloc[:, :768]
y_val_texts = val_texts.iloc[:, 769:] # + val_sentences.iloc[:, 769:]

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

input_dim = X_train_texts.shape[1]
output_dim = y_train_texts.shape[1]

model = Sequential([
    Dense(1024, activation='relu', input_shape=(input_dim, )),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(output_dim, activation='sigmoid')
])

/Users/mchojna/Documents/Repozytoria/emotion-and-sentiment-recognition/.venv/lib/python3.9/site-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-05-28 18:16:17.780079: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-05-28 18:16:17.780344: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-05-28 18:16:17.780674: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-05-28 18:16:17.781158: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-05-28 18:16:17.781455: I tensorflow/core/common_runtime/pluggable

In [8]:
from keras.metrics import Precision, Recall

optimizer = Adam(learning_rate=0.001, decay=1e-6)

model.compile(
    optimizer=optimizer, 
    loss='binary_crossentropy', 
    metrics=[
        'binary_accuracy',
        Precision(thresholds=0.5, name='precision'),
        Recall(thresholds=0.5, name='recall'),
    ]
)

/Users/mchojna/Documents/Repozytoria/emotion-and-sentiment-recognition/.venv/lib/python3.9/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


In [9]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1024)           │       787,456 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 512)            │         2,048 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 256)            │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 11)             │         1,419 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,485,579 (5.67 MB)

 Trainable params: 1,481,739 (5.65 MB)

 Non-trainable params: 3,840 (15.00 KB)

In [10]:
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

history = model.fit(
    X_train_texts, y_train_texts,
    epochs=16,
    batch_size=8,
    validation_data=(X_val_texts, y_val_texts),
    callbacks=[early_stop, reduce_lr],
)

Epoch 1/16


2025-05-28 18:16:22.701245: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


800/800 ━━━━━━━━━━━━━━━━━━━━ 32s 37ms/step - binary_accuracy: 0.7372 - loss: 0.5450 - precision: 0.5284 - recall: 0.6522 - val_binary_accuracy: 0.8603 - val_loss: 0.3316 - val_precision: 0.7817 - val_recall: 0.6939 - learning_rate: 0.0010
Epoch 2/16
800/800 ━━━━━━━━━━━━━━━━━━━━ 28s 35ms/step - binary_accuracy: 0.8443 - loss: 0.3641 - precision: 0.7489 - recall: 0.6307 - val_binary_accuracy: 0.8666 - val_loss: 0.3215 - val_precision: 0.8130 - val_recall: 0.6786 - learning_rate: 0.0010
Epoch 3/16
800/800 ━━━━━━━━━━━━━━━━━━━━ 26s 32ms/step - binary_accuracy: 0.8495 - loss: 0.3566 - precision: 0.7666 - recall: 0.6391 - val_binary_accuracy: 0.8671 - val_loss: 0.3084 - val_precision: 0.8107 - val_recall: 0.6842 - learning_rate: 0.0010
Epoch 4/16
800/800 ━━━━━━━━━━━━━━━━━━━━ 26s 32ms/step - binary_accuracy: 0.8556 - loss: 0.3395 - precision: 0.7771 - recall: 0.6473 - val_binary_accuracy: 0.8650 - val_loss: 0.3149 - val_precision: 0.8086 - val_recall: 0.6770 - learning_rate: 0.0010
Epoch 5/16


In [12]:
model.save('../models/neural_network.keras') 

In [13]:
import keras

history = keras.models.load_model('../models/neural_network.keras')

In [14]:
import numpy as np
from typing import Dict

def evaluate_texts(model, X_test: pd.DataFrame, y_test: pd.DataFrame, threshold: float=0.5) -> Dict:
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > threshold).astype(int)
    
    y_test = y_test.values
    
    labels = ['Joy', 'Trust', 'Anticipation', 'Surprise', 'Fear', 'Sadness', 'Disgust', 'Anger', 'Positive', 'Negative', 'Neutral']
    metrics = {}
    
    metrics['F1-score macro texts'] = 0
    for label in labels:
        metrics[f"Precision {label}"] = 0
        metrics[f"Recall {label}"] = 0
        metrics[f"F1-score {label}"] = 0
        metrics[f"TP {label}"] = 0
        metrics[f"FP {label}"] = 0
        metrics[f"TN {label}"] = 0
        metrics[f"FN {label}"] = 0
    
    for i, label in enumerate(labels):
        metrics[f"TP {label}"] = int(np.sum((y_pred_binary[:, i] == 1) & (y_test[:, i] == 1)))
        metrics[f"FP {label}"] = int(np.sum((y_pred_binary[:, i] == 1) & (y_test[:, i] == 0)))
        metrics[f"TN {label}"] = int(np.sum((y_pred_binary[:, i] == 0) & (y_test[:, i] == 0)))
        metrics[f"FN {label}"] = int(np.sum((y_pred_binary[:, i] == 0) & (y_test[:, i] == 1)))
        
    for label in labels:
        metrics[f"Precision {label}"] = metrics[f"TP {label}"] / (metrics[f"TP {label}"] + metrics[f"FP {label}"] + 1e-8)
        metrics[f"Recall {label}"] = metrics[f"TP {label}"] / (metrics[f"TP {label}"] + metrics[f"FN {label}"] + 1e-8)
        metrics[f"F1-score {label}"] = 2 * (metrics[f"Precision {label}"] * metrics[f"Recall {label}"]) / (metrics[f"Precision {label}"] + metrics[f"Recall {label}"] + 1e-8)

    metrics['F1-score macro texts'] = sum(metrics[f"F1-score {label}"] for label in labels) / len(labels)
    
    metrics = {k: v for (k, v) in metrics.items() if ('Precision' in k) or ('Recall' in k) or ('F1' in k)}

    return metrics

In [15]:
def evaluate_sentences_v1(model, X_test: pd.DataFrame, y_test: pd.DataFrame, threshold: float=0.5) -> Dict:
    y_pred = model.predict(X_test)
    y_pred_binary = (y_pred > threshold).astype(int)
    
    y_test = y_test.values
    
    labels = ['Joy', 'Trust', 'Anticipation', 'Surprise', 'Fear', 'Sadness', 'Disgust', 'Anger', 'Positive', 'Negative', 'Neutral']
    metrics = {}
    
    metrics['F1-score macro sentences'] = 0
    for label in labels:
        metrics[f"Precision {label}"] = 0
        metrics[f"Recall {label}"] = 0
        metrics[f"F1-score {label}"] = 0
        metrics[f"TP {label}"] = 0
        metrics[f"FP {label}"] = 0
        metrics[f"TN {label}"] = 0
        metrics[f"FN {label}"] = 0
    
    for i, label in enumerate(labels):
        metrics[f"TP {label}"] = int(np.sum((y_pred_binary[:, i] == 1) & (y_test[:, i] == 1)))
        metrics[f"FP {label}"] = int(np.sum((y_pred_binary[:, i] == 1) & (y_test[:, i] == 0)))
        metrics[f"TN {label}"] = int(np.sum((y_pred_binary[:, i] == 0) & (y_test[:, i] == 0)))
        metrics[f"FN {label}"] = int(np.sum((y_pred_binary[:, i] == 0) & (y_test[:, i] == 1)))
        
    for label in labels:
        metrics[f"Precision {label}"] = metrics[f"TP {label}"] / (metrics[f"TP {label}"] + metrics[f"FP {label}"] + 1e-8)
        metrics[f"Recall {label}"] = metrics[f"TP {label}"] / (metrics[f"TP {label}"] + metrics[f"FN {label}"] + 1e-8)
        metrics[f"F1-score {label}"] = 2 * (metrics[f"Precision {label}"] * metrics[f"Recall {label}"]) / (metrics[f"Precision {label}"] + metrics[f"Recall {label}"] + 1e-8)

    metrics['F1-score macro sentences'] = sum(metrics[f"F1-score {label}"] for label in labels) / len(labels)

    metrics = {k: v for (k, v) in metrics.items() if ('Precision' in k) or ('Recall' in k) or ('F1' in k)}

    return metrics


In [16]:
def get_hash_indeces(df: pd.DataFrame) -> List:
    hash_indices = []
    for index in df.index.tolist():
        if str(df.loc[index, 'text']).startswith('#'):
            hash_indices.append(index)
          
    result_hash_indices = []   
    i = 0;
    for index in hash_indices:
        index = index - i
        result_hash_indices.append(index)
        i += 1
    
    return result_hash_indices

In [21]:
def evaluate_sentences_v2(model: keras.Model, X_test: pd.DataFrame, y_test: pd.DataFrame, threshold: float=1): # -> Dict:
    raw_data = encode_labels(pd.read_csv('../data/raw/test.csv'))
    hash_indices = get_hash_indeces(raw_data)
    
    y_pred = model.predict(X_test)
    y_test = y_test.values
    
    labels = ['Joy', 'Trust', 'Anticipation', 'Surprise', 'Fear', 'Sadness', 'Disgust', 'Anger', 'Positive', 'Negative', 'Neutral']
    metrics = {}
    
    metrics['F1-score macro sentences'] = 0
    for label in labels:
        metrics[f"Precision {label}"] = 0
        metrics[f"Recall {label}"] = 0
        metrics[f"F1-score {label}"] = 0
        metrics[f"TP {label}"] = 0
        metrics[f"FP {label}"] = 0
        metrics[f"TN {label}"] = 0
        metrics[f"FN {label}"] = 0
    
    y_true_segments = []
    y_pred_segments = []
    
    i = 0
    start = 0
    for index in hash_indices:
        pred_sum = np.zeros(11, dtype=np.float64)
        for y_pred_i in y_pred[start:index]:
            pred_sum += y_pred_i
            
        y_true_segments.append(raw_data.iloc[index+0, 1:].to_numpy())
        y_pred_segments.append((pred_sum >= threshold).astype(int))
                    
        i += 1
        start = index

    y_true_segments = np.array(y_true_segments)
    y_pred_segments = np.array(y_pred_segments)

    for i, label in enumerate(labels):
        metrics[f"TP {label}"] = int(np.sum((y_pred_segments[:, i] == 1) & (y_true_segments[:, i] == 1)))
        metrics[f"FP {label}"] = int(np.sum((y_pred_segments[:, i] == 1) & (y_true_segments[:, i] == 0)))
        metrics[f"TN {label}"] = int(np.sum((y_pred_segments[:, i] == 0) & (y_true_segments[:, i] == 0)))
        metrics[f"FN {label}"] = int(np.sum((y_pred_segments[:, i] == 0) & (y_true_segments[:, i] == 1)))
        
    for label in labels:
        metrics[f"Precision {label}"] = metrics[f"TP {label}"] / (metrics[f"TP {label}"] + metrics[f"FP {label}"] + 1e-8)
        metrics[f"Recall {label}"] = metrics[f"TP {label}"] / (metrics[f"TP {label}"] + metrics[f"FN {label}"] + 1e-8)
        metrics[f"F1-score {label}"] = 2 * (metrics[f"Precision {label}"] * metrics[f"Recall {label}"]) / (metrics[f"Precision {label}"] + metrics[f"Recall {label}"] + 1e-8)

    metrics['F1-score macro sentences'] = sum(metrics[f"F1-score {label}"] for label in labels) / len(labels)

    metrics = {k: v for (k, v) in metrics.items() if ('Precision' in k) or ('Recall' in k) or ('F1' in k)}

    return metrics

In [22]:
texts_metrics = evaluate_texts(history, X_test=test_texts.iloc[:, :768], y_test=test_texts.iloc[:, 769:], threshold=0.5)
print(texts_metrics)

40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
{'F1-score macro texts': 0.5805066691752971, 'Precision Joy': 0.8188539741068603, 'Recall Joy': 0.8054545454399008, 'F1-score Joy': 0.8120989867361404, 'Precision Trust': 0.7341772151434065, 'Recall Trust': 0.5550239234184199, 'F1-score Trust': 0.6321525836179644, 'Precision Anticipation': 0.6962025315574428, 'Recall Anticipation': 0.43999999996480005, 'F1-score Anticipation': 0.5392156814758748, 'Precision Surprise': 0.0, 'Recall Surprise': 0.0, 'F1-score Surprise': 0.0, 'Precision Fear': 0.7999999984, 'Recall Fear': 0.06779661015800058, 'F1-score Fear': 0.12499999852050782, 'Precision Sadness': 0.8539325842536717, 'Recall Sadness': 0.8290909090758347, 'F1-score Sadness': 0.8413284082696996, 'Precision Disgust': 0.6203703703416495, 'Recall Disgust': 0.551440329195414, 'F1-score Disgust': 0.5838779906345613, 'Precision Anger': 0.6874999999386161, 'Recall Anger': 0.36666666664920633, 'F1-score Anger': 0.47826086499864984, 'Precision Positive': 0.82

In [23]:
sentences_metrics_v1 = evaluate_sentences_v1(history, X_test=test_sentences.iloc[:, :768], y_test=test_sentences.iloc[:, 769:], threshold=0.5)
print(sentences_metrics_v1)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
{'F1-score macro sentences': 0.07439066067541268, 'Precision Joy': 0.6363636357851239, 'Recall Joy': 0.08139534882774473, 'F1-score Joy': 0.1443298948666171, 'Precision Trust': 0.0, 'Recall Trust': 0.0, 'F1-score Trust': 0.0, 'Precision Anticipation': 0.0, 'Recall Anticipation': 0.0, 'F1-score Anticipation': 0.0, 'Precision Surprise': 0.0, 'Recall Surprise': 0.0, 'F1-score Surprise': 0.0, 'Precision Fear': 0.0, 'Recall Fear': 0.0, 'F1-score Fear': 0.0, 'Precision Sadness': 0.5999999996, 'Recall Sadness': 0.09890109889023066, 'F1-score Sadness': 0.16981131829298685, 'Precision Disgust': 0.24999999968749997, 'Recall Disgust': 0.0444444444345679, 'F1-score Disgust': 0.075471695521538, 'Precision Anger': 0.999999995, 'Recall Anger': 0.0444444444345679, 'F1-score Anger': 0.08510638212765959, 'Precision Positive': 0.5833333328472222, 'Recall Positive': 0.07216494844616857, 'F1-score Positive': 0.128440364989479, 'Precision Negative': 0.5624999996484374

In [24]:
sentences_metrics_v2 = evaluate_sentences_v2(history, X_test=test_sentences.iloc[:, :768], y_test=test_sentences.iloc[:, 769:], threshold=0.5)
print(sentences_metrics_v2)

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
{'F1-score macro sentences': 0.15606171562838356, 'Precision Joy': 0.479999999808, 'Recall Joy': 0.16901408448323746, 'F1-score Joy': 0.24999999609592016, 'Precision Trust': 0.23809523798185941, 'Recall Trust': 0.19999999992, 'F1-score Trust': 0.21739129929111542, 'Precision Anticipation': 0.07142857137755101, 'Recall Anticipation': 0.0624999999609375, 'F1-score Anticipation': 0.06666666164444482, 'Precision Surprise': 0.0, 'Recall Surprise': 0.0, 'F1-score Surprise': 0.0, 'Precision Fear': 0.0, 'Recall Fear': 0.0, 'F1-score Fear': 0.0, 'Precision Sadness': 0.5555555553497942, 'Recall Sadness': 0.19230769228303748, 'F1-score Sadness': 0.28571428183945585, 'Precision Disgust': 0.13636363630165288, 'Recall Disgust': 0.09999999996666667, 'F1-score Disgust': 0.11538461045858009, 'Precision Anger': 0.14285714278911565, 'Recall Anger': 0.0909090908815427, 'F1-score Anger': 0.11111110631687264, 'Precision Positive': 0.519999999792, 'Recall Positive': 0.

In [ ]:
def calculate_final_score(text_metrics: Dict, sentences_metrics: Dict) -> float:
    return (text_metrics['F1-score macro texts'] + sentences_metrics['F1-score macro sentences']) / 2

In [ ]:
calculate_final_score(texts_metrics, sentences_metrics_v1)

0.3590148730180372

In [ ]:
calculate_final_score(texts_metrics, sentences_metrics_v2)

0.408699973106216